In [136]:
#loading the libraries
import os
import cv2
import scipy
from scipy import optimize
from scipy.optimize import minimize
import numpy as np
from scipy.io import loadmat
import scipy.misc as smp
import random
import pandas as pd
os.chdir("C:/Users/Pranshul/Desktop/machine-learning-ex4/ex4");

In [156]:
img_path = 'C:/Users/Pranshul/Desktop/Machine-Learning/1.jpeg';
img = cv2.imread(img_path, 0);
img_reverted= cv2.bitwise_not(img)
img_reverted = np.reshape(img_reverted,(1,400),order='C');
img_reverted.shape

(1, 400)

In [147]:
#loading the data
data = loadmat("ex4data1.mat");
nn_params = loadmat("ex4weights.mat");
Theta1 = np.array(nn_params['Theta1']);
Theta2 = np.array(nn_params['Theta2']);
x = np.array(data['X']);
y = np.array(data['y']);
input_layer_size  = 400;
hidden_layer_size = 25;
num_labels = 10;
m =x.shape[0];
theta1 = Theta1.flatten();
theta2 = Theta2.flatten();
k = 10;
nn_params = np.concatenate((theta1,theta2),axis = 0);
size1 = theta1.shape[0]
size2 = theta2.shape[0]
Theta1.shape

(25, 401)

In [148]:
lamda = 0;
def sigmoid(z):
    g = 1/(1 + np.exp(-z));
    return g;

In [149]:
#Cost function
def costfunction(nn_params, input_layer_size, hidden_layer_size,num_labels, x, y,lamda):
    Theta1 = np.zeros((hidden_layer_size, input_layer_size + 1));
    Theta2 = np.zeros((num_labels, hidden_layer_size + 1));
    Theta1 = np.reshape(nn_params[0:int(size1)],(Theta1.shape),order='C')
    Theta2 = np.reshape(nn_params[int(size1):int(size1+size2)],(Theta2.shape),order='C')
    yvec = np.zeros((m,k));
    for i in range(0,m):
        yvec[i][int(y[i])-1] = 1;
    Theta1 = np.reshape(Theta1,(hidden_layer_size,(input_layer_size+1)),order='C');
    Theta2 = np.reshape(Theta2,(num_labels,(hidden_layer_size+1)),order='C');
    #calculating the hypothesis
    a1 = x;
    q1 = np.ones((x.shape[0],1));
    a1 = np.append(q1, a1, axis=1);
    z1 = np.dot(a1,np.transpose(Theta1));
    q2 = np.ones((x.shape[0],1));
    z1 = np.append(q2, z1, axis=1);
    a2 = sigmoid(z1);
    a3 = sigmoid(np.dot(a2,np.transpose(Theta2)));
    h = a3;        
    #calculating the regularization term
    Reg = np.sum(np.dot(Theta1,np.transpose(Theta1))) + np.sum(np.dot(Theta2,np.transpose(Theta2))) ; 
    #calculating the cost
    J = 0;
    J = np.sum((-(yvec)*(np.log(h)) - (np.ones(yvec.shape)-yvec)*np.log(np.ones(h.shape) - h)));
    J = J/m + lamda*Reg/(2*m);
    #calculating the grad
    q1 = np.zeros((25,401));
    q2 = np.zeros((10,26));
    for t in range(0,m):
        a1 = x[t,:];
        a1 = np.append([1], a1, axis=0);
        a1 = np.reshape(a1,(int(a1.shape[0]),1),order='C');
        z1 = np.dot(Theta1,a1);
        a2 = sigmoid(z1);
        w = np.reshape(np.array([1]),(1,1),order='C')
        a2 = np.append(w, a2, axis=0);
        z2 = np.dot(Theta2,a2);
        a3 = sigmoid(z2);
        o = yvec[t,:];
        o = np.reshape(o,(int(o.shape[0]),1),order='C');
        delta3 = np.subtract(a3,o);
        delta2 = np.dot(np.transpose(Theta2),delta3) * sigmoidGradient(np.append(w,z1,axis=0));
        delta2 = delta2[1:26,:];
        q1 = q1 + np.dot(delta2,np.transpose(a1));
        q2 = q2 + np.dot(delta3,np.transpose(a2));
    q1 = q1/m;
    q2 = q2/m;
    q1 = np.reshape(q1,(q1.shape[0]*q1.shape[1],1),order='C');
    q2 = np.reshape(q2,(q2.shape[0]*q2.shape[1],1),order='C');
    grad = np.append(q1,q2,axis=0);

    return J,grad;    

In [150]:
def sigmoidGradient(z):
    g = 1/(1 + np.exp(-z));
    g = g*(1-g);
    return g;

In [151]:
initial_Theta1 = np.random.randint(-1,1,size=(hidden_layer_size, input_layer_size + 1));
initial_Theta2 = np.random.randint(-1,1,size=(hidden_layer_size, num_labels+1));
initial_Theta1 = np.reshape(initial_Theta1,(initial_Theta1.shape[0]*initial_Theta1.shape[1],1),order='C');
initial_Theta2 = np.reshape(initial_Theta2,(initial_Theta2.shape[0]*initial_Theta2.shape[1],1),order='C');
initial_nn_params = np.append(initial_Theta1,initial_Theta2,axis=0);
nn_params.shape

(10285,)

In [155]:
maxiter = 20
lambda_reg = 0.1
myargs = (input_layer_size, hidden_layer_size, num_labels, x,y, lambda_reg)
results = minimize(costfunction, x0=nn_params, args=myargs, options={'disp': True, 'maxiter':maxiter}, method="L-BFGS-B", jac=True);
nn_params = results["x"]
Theta1 = np.reshape(nn_params[0:int(size1)],(Theta1.shape),order='C')
Theta2 = np.reshape(nn_params[int(size1):int(size1+size2)],(Theta2.shape),order='C');

array([[-4.20492876e-02, -1.05624163e-08,  2.19414684e-09, ...,
        -1.09975244e-04, -1.64974267e-05,  2.80464449e-09],
       [ 4.89041895e-01,  7.66168682e-09, -9.75873689e-09, ...,
        -2.93617549e-04,  6.89579610e-07,  3.54422854e-09],
       [ 1.37777201e+00, -8.77654466e-09,  8.16037764e-09, ...,
        -4.32972145e-04, -7.75452355e-06, -7.50668099e-09],
       ...,
       [-7.13058243e-01, -8.89272060e-09, -9.81968100e-09, ...,
         7.71888426e-05, -1.07282044e-05,  9.02499060e-09],
       [-1.93070761e+00,  3.05178374e-10,  2.56061008e-09, ...,
        -3.08212675e-03,  3.38991651e-04,  3.83761998e-09],
       [-1.47932571e+00,  8.85876862e-09, -6.57515140e-10, ...,
        -6.05074571e-05, -2.56087203e-05,  8.79454531e-09]])

In [157]:
#predicting the number
def predict(x,Theta1,Theta2):
    p = np.zeros((m,1))
    a1 = x;
    q1 = np.ones((x.shape[0],1));
    a1 = np.append(q1, a1, axis=1);
    z1 = np.dot(a1,np.transpose(Theta1));
    q2 = np.ones((x.shape[0],1));
    z1 = np.append(q2, z1, axis=1);
    a2 = sigmoid(z1);
    a3 = sigmoid(np.dot(a2,np.transpose(Theta2)));
    h = a3; 
    p = np.argmax(h, axis=1)
    return p+1;

In [170]:
a = predict(x,Theta1,Theta2)
a = np.reshape(a,(5000,1),order='C');
accuracy = np.sum(a==y)/m*100;
accuracy

99.94